# Developing Merge Utilities

## Setup

In [1]:
import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import urllib
import datetime
import math
import shapely
import xarray as xr
import matplotlib.pyplot as plt
from io import BytesIO, StringIO
import scipy.stats as stats

import s3fs
import tempfile # Used for downloading (and then deleting) netcdfs to local drive from s3 bucket
import os
from shapely.geometry import Point
import time # Used for progress bar 
import sys # Used for progress bar 

# Silence warnings
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) # Warning is raised when creating Point object from coords. Can't figure out why. 

In [2]:
## Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client('s3') # for lower-level processes

## Set relative paths to other folders and objects in repository.
bucket_name = "wecc-historical-wx"
wecc_terr = "s3://wecc-historical-wx/0_maps/WECC_Informational_MarineCoastal_Boundary_land.shp"
wecc_mar = "s3://wecc-historical-wx/0_maps/WECC_Informational_MarineCoastal_Boundary_marine.shp"

In [3]:
# Define temporary directory in local drive for downloading data from S3 bucket
# If the directory doesn't exist, it will be created
# If we used zarr, this wouldn't be neccessary 
temp_dir = "./tmp"
if not os.path.exists(temp_dir): 
    os.mkdir(temp_dir)

## Define helper functions

In [4]:
def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.6+
    """
    Print a progress bar to console 

    References
    ----------
    https://stackoverflow.com/questions/3160699/python-progress-bar
    
    """
    count = len(it)
    start = time.time() # time estimate start
    def show(j):
        x = int(size*j/count)
        # time estimate calculation and string
        remaining = ((time.time() - start) / j) * (count - j)        
        mins, sec = divmod(remaining, 60) # limited to minutes
        time_str = f"{int(mins):02}:{sec:03.1f}"
        print(f"{prefix}[{u'█'*x}{('.'*(size-x))}] {j}/{count} Est wait {time_str}", end='\r', file=out, flush=True)
    show(0.1) # avoid div/0 
    for i, item in enumerate(it):
        yield item
        show(i+1)
    print("\n", flush=True, file=out)

In [5]:
def read_nc_from_s3(network_name, station_id, temp_dir):
    """Read netcdf file containing station data for a single station of interest from AWS s3 bucket 

    Parameters
    ----------
    network_name: str 
        Name of network (i.e. "ASOSAWOS")
        Must correspond with a valid directory in the s3 bucket (i.e. "CAHYDRO", "CDEC", "ASOSAWOS")
    station_id: str
        Station identifier; i.e. the name of the netcdf file in the bucket (i.e. "ASOSAWOS_72012200114.nc")
    
    Returns 
    -------
    station_data: xr.Dataset 
    
    Notes
    -----
    The data is first downloaded from AWS into a tempfile, which is then deleted after xarray reads in the file 
    I'd like to see us use a zarr workflow if possible to avoid this. 

    """

    # Temp file for downloading from s3
    temp_file = tempfile.NamedTemporaryFile(
        dir = temp_dir, 
        prefix = "", 
        suffix = ".nc",
        delete = True
    )

    # Create s3 file system 
    s3 = s3fs.S3FileSystem(anon=False)

    # Get URL to netcdf in S3
    s3_url = 's3://wecc-historical-wx/3_qaqc_wx_dev/{}/{}.nc'.format(network_name, station_id)

    # Read in the data using xarray 
    s3_file_obj = s3.get(s3_url, temp_file.name)
    station_data = xr.open_dataset(temp_file.name, engine='h5netcdf').load()

    # Close temporary file 
    temp_file.close()

    return station_data 

In [6]:
def qaqc_ds_to_df(ds, verbose=False):
    ## Add qc_flag variable for all variables, including elevation; 
    ## defaulting to nan for fill value that will be replaced with qc flag

    for key,val in ds.variables.items():
        if val.dtype==object:
            if key=='station':
                if str in [type(v) for v in ds[key].values]:
                    ds[key] = ds[key].astype(str)
            else:
                if str in [type(v) for v in ds.isel(station=0)[key].values]:
                    ds[key] = ds[key].astype(str)
                
    exclude_qaqc = ["time", "station", "lat", "lon", 
                    "qaqc_process", "sfcWind_method", 
                    "pr_duration", "pr_depth", "PREC_flag",
                    "rsds_duration", "rsds_flag", 
                    "anemometer_height_m",
                    "thermometer_height_m"
                   ] # lat, lon have different qc check

    raw_qc_vars = [] # qc_variable for each data variable, will vary station to station
    era_qc_vars = [] # our ERA qc variable
    old_era_qc_vars = [] # our ERA qc variable

    for var in ds.data_vars:
        if 'q_code' in var: 
            raw_qc_vars.append(var) # raw qc variable, need to keep for comparison, then drop
        if '_qc' in var: 
            raw_qc_vars.append(var) # raw qc variables, need to keep for comparison, then drop
        if '_eraqc' in var:
            era_qc_vars.append(var) # raw qc variables, need to keep for comparison, then drop
            old_era_qc_vars.append(var)

    print(f"era_qc existing variables:\n{era_qc_vars}")
    n_qc = len(era_qc_vars)
    
    for var in ds.data_vars:
        if var not in exclude_qaqc and var not in raw_qc_vars and "_eraqc" not in var:
            qc_var = var + "_eraqc" # variable/column label

            # if qaqc var does not exist, adds new variable in shape of original variable with designated nan fill value
            if qc_var not in era_qc_vars:
                print(f"nans created for {qc_var}")
                ds = ds.assign({qc_var: xr.ones_like(ds[var])*np.nan})
                era_qc_vars.append(qc_var)
    
    print("{} created era_qc variables".format(len(era_qc_vars)-len(old_era_qc_vars)))
    if len(era_qc_vars)!=n_qc:    
        print("{}".format(np.setdiff1d(old_era_qc_vars, era_qc_vars)))
    
    # Save attributes to inheret them to the QAQC'ed file
    attrs = ds.attrs
    var_attrs = {var:ds[var].attrs for var in list(ds.data_vars.keys())}

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        df = ds.to_dataframe()

    # instrumentation heights
    if 'anemometer_height_m' not in df.columns:
        try:
            df['anemometer_height_m'] = np.ones(ds['time'].shape)*ds.anemometer_height_m
        except:
            print("Filling anemometer_height_m with NaN.", flush=True)
            df['anemometer_height_m'] = np.ones(len(df))*np.nan
        finally:
            pass
    if 'thermometer_height_m' not in df.columns:
        try:
            df['thermometer_height_m'] = np.ones(ds['time'].shape)*ds.thermometer_height_m
        except:
            print("Filling thermometer_height_m with NaN.", flush=True)
            df['thermometer_height_m'] = np.ones(len(df))*np.nan
        finally:
            pass

    # De-duplicate time axis
    df = df[~df.index.duplicated()].sort_index()
           
    # Save station/time multiindex
    MultiIndex = df.index
    station = df.index.get_level_values(0)
    df['station'] = station
    
    # Station pd.Series to str
    station = station.unique().values[0]
    
    # Convert time/station index to columns and reset index
    df = df.droplevel(0).reset_index()

    # Add time variables needed by multiple functions
    df['hour'] = pd.to_datetime(df['time']).dt.hour
    df['day'] = pd.to_datetime(df['time']).dt.day 
    df['month'] = pd.to_datetime(df['time']).dt.month 
    df['year'] = pd.to_datetime(df['time']).dt.year 
    df['date']  = pd.to_datetime(df['time']).dt.date
    
    return df#, MultiIndex, attrs, var_attrs, era_qc_vars 

## Precipitation Standardization

### load and explore sample precipication data

In [7]:
# load in single dc file from AWS
ds = read_nc_from_s3('ASOSAWOS', 'ASOSAWOS_72494023234', temp_dir)
#convert to formatted pandas dataframe
df = qaqc_ds_to_df(ds, verbose=False)

# check if precipittion data present
all_pr_vars = [var for var in df.columns if 'pr' in var]
all_pr_vars


# ASOSAWOS no precip
# CAHYDOR: ['pr', 'pr_eraqc']
# CDEC: ['pr', 'pr_eraqc']
# CIMIS: ['pr', 'pr_qc', 'pr_eraqc']
# CNREFC:  ['pr', 'pr_qc', 'pr_eraqc']
# CRN: ['pr_5min', 'pr_5min_qc', 'pr_1h', 'pr_5min_eraqc', 'pr_1h_eraqc']
# CWOP: no precip


era_qc existing variables:
['ps_eraqc', 'tas_eraqc', 'tdps_eraqc', 'pr_eraqc', 'sfcWind_eraqc', 'sfcWind_dir_eraqc', 'elevation_eraqc', 'ps_altimeter_eraqc', 'psl_eraqc']
0 created era_qc variables


['pr', 'qaqc_process', 'pr_qc', 'pr_duration', 'pr_depth_qc', 'pr_eraqc']

In [11]:
test = df[['time','pr', 'qaqc_process', 'pr_qc', 'pr_duration', 'pr_depth_qc', 'pr_eraqc']] 
test

,time,pr,qaqc_process,pr_qc,pr_duration,pr_depth_qc,pr_eraqc
0,1980-01-01 00:00:00,0.0,V020,5,0 days 01:00:00,9.0,NaN
1,1980-01-01 00:28:00,NaN,V020,9,0 days 06:00:00,9.0,NaN
2,1980-01-01 01:00:00,0.0,V020,5,0 days 01:00:00,9.0,NaN
3,1980-01-01 01:14:00,NaN,V020,,NaT,NaN,NaN
4,1980-01-01 02:00:00,0.0,V020,5,0 days 01:00:00,9.0,NaN
...,...,...,...,...,...,...,...
464372,2022-08-31 19:56:00,0.0,V030,5,0 days 01:00:00,9.0,NaN
464373,2022-08-31 20:56:00,0.0,V030,5,0 days 01:00:00,9.0,NaN
464374,2022-08-31 21:56:00,0.0,V030,5,0 days 01:00:00,9.0,NaN
464375,2022-08-31 22:56:00,0.0,V030,5,0 days 01:00:00,9.0,NaN


In [12]:
result = test.resample('1h',on='time').sum()
result

,pr,qaqc_process,pr_qc,pr_duration,pr_depth_qc,pr_eraqc
time,,,,,,
1980-01-01 00:00:00,0.0,V020V020,59,0 days 07:00:00,18.0,0.0
1980-01-01 01:00:00,0.0,V020V020,5,0 days 01:00:00,9.0,0.0
1980-01-01 02:00:00,0.0,V020,5,0 days 01:00:00,9.0,0.0
1980-01-01 03:00:00,0.0,V020,5,0 days 01:00:00,9.0,0.0
1980-01-01 04:00:00,0.0,V020,5,0 days 01:00:00,9.0,0.0
...,...,...,...,...,...,...
2022-08-31 19:00:00,0.0,V030,5,0 days 01:00:00,9.0,0.0
2022-08-31 20:00:00,0.0,V030,5,0 days 01:00:00,9.0,0.0
2022-08-31 21:00:00,0.0,V030,5,0 days 01:00:00,9.0,0.0


In [13]:
def precip_hourly_standardization(df):
    
    """
    
    Rules:
    ------
        1) pr_5min < pr_1h < pr_24h
        2) pr_localmid should never exceed pr_24h

    Input:
    ------
        df [pd.DataFrame]: station dataset converted to dataframe through QAQC pipeline

    Output:
    -------
        if success:
            df [pd.DataFrame]: QAQC dataframe with additional column "pr_1hr"
        if failure:
            None
    """

    print("Running: precip_hourly_standardization", flush=True)

    metadata = ['time', 'tas', 'pr_5min', 'pr_5min_qc', 'tas_qc', 'pr_1h', 'tas_eraqc', 'pr_5min_eraqc', 'pr_1h_eraqc']
    observations =[] 

    # identify which precipitation vars are reported by a station
    vars_to_remove = ['qc', 'duration', 'method', 'depth']
    all_pr_vars = [var for var in df.columns if 'pr' in var] # can be variable length depending if there is a raw qc var
    pr_vars = [var for var in all_pr_vars if not any(True for item in vars_to_remove if item in var)] # remove all qc variables so they do not also run through: raw, eraqc, qaqc_process
    
    # TODO: add to log file
    try:        
        # if station does not report any precipitation values, or only one, bypass
        if len(pr_vars) == 0:
            print('Station does not report precipitation variables - bypassing hourly aggregation', flush=True)
            return df
        if 'pr_1h' in pr_vars and 'pr_5min' not in pr_vars:
            print('Station already reports hourly precipitation - bypassing hourly aggregation', flush=True)
            return df
        if 'pr_24h' in pr_vars and 'pr_5min' not in pr_vars:
            print('Station reports daily precipitation - bypassing hourly aggregation', flush=True)
            return df
        if 'pr_5min' in pr_vars and 'pr_24h' not in pr_vars and 'pr_1h' not in pr_vars:
            print('attempting hourly aggregation', flush=True)
            df = df.resample('1h',on='time').sum()
            return df
    
    except Exception as e:
        print("precip_hourly_standardization failed with Exception: {0}".format(e), flush=true)
        return None

In [ ]:
result = precip_hourly_standardization(df)
result

Running: precip_hourly_standardization
